In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Imports

In [2]:
# import necessary libraries
import mesa
import seaborn as sns
import pandas as pd
import shutil

# import cacheable model wrapper
from mesa.cacheable_model import CacheableModel

### Output directory 
"output_dir" is the path which the cached data is stored. The directory is included in the .gitignore file. 

In [3]:
directory = "output_dir"

def delete_directory(directory_path):
    shutil.rmtree(directory_path)
    print(f"Directory {directory_path} and its contents have been deleted.")

try:
    delete_directory(directory)
except:
    pass

Directory output_dir and its contents have been deleted.


### Base Model
Define the Boltzmann Wealth Model. I chose to reimplement it here so that I can modify the attributes and experiment with the results more easily. The code is taken from the tutorial.

In [4]:
def compute_gini(model):
    agent_wealths = [agent.wealth for agent in model.schedule.agents]
    x = sorted(agent_wealths)
    N = model.num_agents
    B = sum(xi * (N - i) for i, xi in enumerate(x)) / (N * sum(x))
    return 1 + (1 / N) - 2 * B


class MoneyAgent(mesa.Agent):
    """An agent with fixed initial wealth."""

    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.wealth = 1

    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos, moore=True, include_center=False
        )
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def give_money(self):
        cellmates = self.model.grid.get_cell_list_contents([self.pos])
        cellmates.pop(
            cellmates.index(self)
        )  # Ensure agent is not giving money to itself
        if len(cellmates) > 1:
            other = self.random.choice(cellmates)
            other.wealth += 1
            self.wealth -= 1
            if other == self:
                print("I JUST GAVE MONEY TO MYSELF HEHEHE!")

    def step(self):
        self.move()
        if self.wealth > 0:
            self.give_money()


class MoneyModel(mesa.Model):
    """A model with some number of agents."""

    def __init__(self, N, width, height):
        super().__init__()
        self.num_agents = N
        self.grid = mesa.space.MultiGrid(width, height, True)
        self.schedule = mesa.time.RandomActivation(self)

        # Create agents
        for i in range(self.num_agents):
            a = MoneyAgent(i, self)
            self.schedule.add(a)
            # Add the agent to a random grid cell
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))

        self.datacollector = mesa.DataCollector(
            model_reporters={"Gini": compute_gini}, agent_reporters={"Wealth": "wealth"}
        )

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()

### Instantiate the base model and cacheable model

In [5]:
model = MoneyModel(100, 10, 10)
number_of_steps = 1000
def condition_function(model_vars): # condition to cache the results specifically
        return model_vars.get('Gini', 0)[-1] > 0.7
cacheable_model = CacheableModel(model, directory, number_of_steps, 100, condition_function=condition_function)

### Run cacheable model and read cached file

In [11]:
for i in range(number_of_steps):
    cacheable_model.model.step()
    cacheable_model.cache()
    cacheable_model.get_grid_dataframe()

In [12]:
model_df, agent_df = cacheable_model.combine_dataframes()
model_df 
agent_df

,Gini
0,0.0000
1,0.3610
2,0.4966
3,0.5262
4,0.5284
...,...
995,0.6852
996,0.6626
997,0.6626
998,0.6324


Wealth
Step AgentID        
0    0             1
     1             1
     2             1
     3             1
     4             1
...              ...
999  95            0
     96            2
     97            2
     98            4
     99            0

[100000 rows x 1 columns]

### Read the critical results dataframe. 
In this simulation, the condition to cache is when gini coefficient > 0.7.

In [13]:
# Replace 'your_file.parquet' with the path to your Parquet file
df = pd.read_parquet('output_dir/special_results.parquet', engine='pyarrow')
df.set_index("Step")


,Gini
Step,
112,0.7084
113,0.7010
114,0.7024
115,0.7164
116,0.7044
120,0.7016
122,0.7052
161,0.7026
207,0.7052


### Cache grid pos and cached grid visualisation

In [14]:
def agent_portrayal(agent):
    size = 10
    color = "tab:red"
    if agent.wealth > 0:
        size = 50
        color = "tab:blue"
    return {"size": size, "color": color}

model_params = {
    "N": {
        "type": "SliderInt",
        "value": 50,
        "label": "Number of agents:",
        "min": 10,
        "max": 100,
        "step": 1,
    },
    "width": 10,
    "height": 10,
}

In [15]:
from matplotlib.figure import Figure
import solara

def make_histogram(model):
    # Note: you must initialize a figure using this method instead of
    # plt.figure(), for thread safety purpose
    fig = Figure()
    ax = fig.subplots()
    wealth_vals = [agent.wealth for agent in model.schedule.agents]
    # Note: you have to use Matplotlib's OOP API instead of plt.hist
    # because plt.hist is not thread-safe.
    ax.hist(wealth_vals, bins=10)
    solara.FigureMatplotlib(fig)

In [16]:
class TestModel(mesa.Model):
    """A model with some number of agents."""

    def __init__(self, N, width, height):
        super().__init__()
        # self.num_agents = N
        # self.grid = mesa.space.MultiGrid(1,1, True)
        # self.schedule = mesa.time.RandomActivation(self)

        # Create agents
        # for i in range(self.num_agents):
            # a = MoneyAgent(i, self)
            # self.schedule.add(a)
            # Add the agent to a random grid cell
            # x = self.random.randrange(self.grid.width)
            # y = self.random.randrange(self.grid.height)
            # self.grid.place_agent(a, (x, y))

        # self.datacollector = mesa.DataCollector()

    def step(self):
        # self.datacollector.collect(self)
        self._steps += 1

In [17]:
from mesa.visualization_caching.solara_viz import SolaraViz
from mesa import Model
from mesa_models.boltzmann_wealth_model.model import BoltzmannWealthModel
page = SolaraViz(
    TestModel,
    model_params,
    measures=["Gini"],
    name="Money Model",
    agent_portrayal=agent_portrayal,
)
# This is required to render the visualization in the Jupyter notebook
page

output_dir/grid_data_001.parquet
output_dir/grid_data_001.parquet


Cannot show ipywidgets in text

In [18]:

def _draw_grid(space, space_ax, agent_portrayal):
    def portray(g):
        x = []
        y = []
        s = []  # size
        c = []  # color
        for i in range(g.width):
            for j in range(g.height):
                content = g._grid[i][j]
                if not content:
                    continue
                if not hasattr(content, "__iter__"):
                    # Is a single grid
                    content = [content]
                for agent in content:
                    data = agent_portrayal(agent)
                    x.append(i)
                    y.append(j)
                    if "size" in data:
                        s.append(data["size"])
                    if "color" in data:
                        c.append(data["color"])
        out = {"x": x, "y": y}
        # This is the default value for the marker size, which auto-scales
        # according to the grid area.
        out["s"] = (180 / max(g.width, g.height)) ** 2
        if len(s) > 0:
            out["s"] = s
        if len(c) > 0:
            out["c"] = c
        return out

    space_ax.set_xlim(-1, space.width)
    space_ax.set_ylim(-1, space.height)
    space_ax.scatter(**portray(space))

In [19]:
space = cacheable_model.reconstruct_grid("output_dir/grid_data_001.parquet")
space_fig = Figure()
space_ax = space_fig.subplots()
_draw_grid(space, space_ax, agent_portrayal)
solara.FigureMatplotlib(space_fig, format="png")

Cannot show ipywidgets in text

In [20]:
!git branch

  3.0a1
  3.0a1-cache-critical-results
  3.0a1-cache-datacollector
  3.0a1-cache-grid-pos
* 3.3.0a1-cache-continuous-space
  main
  tutorials


# New caching method

In [35]:
model = MoneyModel(100, 10, 10)
number_of_steps = 1000
def condition_function(model_vars): # condition to cache the results specifically
        return model_vars.get('Gini', 0)[-1] > 0.7
cacheable_model = CacheableModel(model, directory, number_of_steps, 1000, condition_function=condition_function)

In [46]:
from mesa.cacheable_model import CacheableModel
parameters = ['unique_id','pos','wealth','width']

for i in range(1000):
    cacheable_model.model.step()
    cacheable_model.get_data(parameters)

KeyError: 'width'

In [40]:
import pyarrow.parquet as pq

parquet_file_path = 'output_dir/grid_data_002.parquet'

table = pq.read_table(parquet_file_path)

df = table.to_pandas()

# Display the DataFrame
df

,unique_id,pos,wealth
0,0,"[1, 5]",0
1,1,"[5, 3]",1
2,2,"[3, 4]",2
3,3,"[6, 7]",0
4,4,"[3, 3]",2
...,...,...,...
95,95,"[2, 4]",0
96,96,"[6, 3]",2
97,97,"[5, 6]",1
98,98,"[7, 2]",1


In [42]:
reconstructed_model = mesa.Model()
column_list = df.columns
agent_list = []
for idx, row in df.iterrows():
    agent = MoneyAgent(None, reconstructed_model)
    for column in column_list:
        setattr(agent, column, row[column])
    agent_list.append(agent)
agent_list

In [45]:
_draw_grid(model, space_ax, agent_portrayal)

AttributeError: 'MoneyModel' object has no attribute 'width'

In [49]:
model.grid.__dict__

{'height': 10,
 'width': 10,
 'torus': True,
 'num_cells': 100,
 '_grid': [[[<__main__.MoneyAgent at 0x7f30e94c5b10>,
   [],
   [],
   []],
  [[],
   [],
   []],
  [[<__main__.MoneyAgent at 0x7f30e94c6920>],
   [],
   [],
   [],
   [],
   [],
  [[<__main__.MoneyAgent at 0x7f30e94e4070>,
   [],
   [],
   [],
   [],
  [[<__main__.MoneyAgent at 0x7f30e94c5930>],
   [],
   [],
  [[<__main__.MoneyAgent at 0x7f30e94c7ca0>],
   [],
   [],
   []],
  [[<__main__.MoneyAgent at 0x7f30e94c6230>,
   [],
   [],
  [[<__main__.MoneyAgent at 0x7f30e94e4550>,
   [],
   [],
   [],
   [],
  [[],
   [],
   [],
   [],
   [],
  [[<__main__.MoneyAgent at 0x7f30e94c5b70>],
   [],
   [],
   [],
   [],
 '_empties_built': False,
 '_neighborhood_cache': {((3, 4), True, False, 1): ((2, 3),
   (2, 4),
   (2, 5),
   (3, 3),
   (3, 5),
   (4, 3),
   (4, 4),
   (4, 5)),
  ((8, 3), True, False, 1): ((7, 2),
   (7, 3),
   (7, 4),
   (8, 2),
   (8, 4),
   (9, 2),
   (9, 3),
   (9, 4)),
  ((5, 1), True, False, 1): ((4, 0),